In [ ]:
from dotenv import load_dotenv
import os

# Cargar variables desde el archivo .env
load_dotenv()

# Ejemplo de cómo acceder a una variable llamada GOOGLE_API_KEY
API_KEY = os.getenv("GOOGLE_API_KEY")

In [7]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point

# Cargar el archivo de barrios directamente desde su carpeta relativa
barrios = gpd.read_file("../data/Barrios/BARRIOS.shp")

# Visualizamos los nombres de las columnas y el primer registro
print(barrios.columns)
barrios.head(1)

Index(['Shape_Leng', 'Shape_Area', 'CODDIS', 'COD_DIS_TX', 'NOMDIS', 'COD_BAR',
       'COD_DISBAR', 'COD_DISB', 'NUM_BAR', 'NOMBRE', 'BARRIO_MAY',
       'BARRIO_MT', 'FCH_ALTA', 'FCH_BAJA', 'OBSERVACIO', 'APROBACION',
       'geometry'],
      dtype='object')


,Shape_Leng,Shape_Area,CODDIS,COD_DIS_TX,NOMDIS,COD_BAR,COD_DISBAR,COD_DISB,NUM_BAR,NOMBRE,BARRIO_MAY,BARRIO_MT,FCH_ALTA,FCH_BAJA,OBSERVACIO,APROBACION,geometry
0,5754.828568,1.469906e+06,1,01,Centro,011,11,1-1,1,Palacio,PALACIO,PALACIO,NaT,NaT,None,None,"POLYGON ((440112.785 4474645.921, 440078.075 4..."


In [8]:
# Convertimos los barrios a GeoJSON (opcional, si lo necesitas luego)
barrios_json = barrios.to_json()
barrios_geojson = json.loads(barrios_json)

In [9]:
import requests
import pandas as pd
import time

def get_coordinates(postal_code):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': f'{postal_code}, Madrid, Spain', 'key': API_KEY}
    response = requests.get(url, params=params).json()
    
    if response['status'] != 'OK':
        raise ValueError(f"Error al obtener coordenadas para {postal_code}: {response['status']}")
    
    result = response['results'][0]
    
    # Validar que el país devuelto sea España mediante short_name = 'ES'
    components = result['address_components']
    country_codes = [c['short_name'] for c in components if 'country' in c['types']]
    if 'ES' not in country_codes:
        raise ValueError(f"Código postal {postal_code} no está en España (devuelto: {country_codes})")
    
    location = result['geometry']['location']
    return location['lat'], location['lng']

def get_places(lat, lng, radius=2000, business_type=None):
    places = []
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'key': API_KEY,
        'type': business_type
    }

    while True:
        res = requests.get(url, params=params).json()
        places.extend(res.get('results', []))
        if 'next_page_token' in res:
            time.sleep(2)
            params['pagetoken'] = res['next_page_token']
        else:
            break

    return places

def clean_data(places, postal_code):
    data = []
    for p in places:
        location = p.get('geometry', {}).get('location', {})
        data.append({
            'nombre': p.get('name'),
            'direccion': p.get('vicinity'),
            'codigo_postal': postal_code,
            'puntuacion_media': p.get('rating'),
            'numero_reviews': p.get('user_ratings_total'),
            'tipo_negocio': ', '.join(p.get('types', [])),
            'latitud': location.get('lat'),
            'longitud': location.get('lng')
        })
    return pd.DataFrame(data)

In [ ]:
from pathlib import Path
import time
import pandas as pd

codigos_postales = ['28001', '28002', '28003', '28004', '28005', '28006', '28007', '28008', '28009',
                    '28010', '28011', '28012', '28013', '28014', '28015', '28016', '28017', '28018', '28019',
                    '28020', '28021', '28022', '28023', '28024', '28025', '28026', '28027', '28028', '28029',
                    '28030', '28031', '28032', '28033', '28034', '28035', '28036', '28037', '28038', '28039',
                    '28040', '28041', '28042', '28043', '28044', '28045', '28046', '28047', '28048', '28049',
                    '28050', '28051', '28052', '28053', '28054', '28055'
                    ]
tipo_negocio = None  # Puedes especificar un tipo como 'restaurant', 'store', etc.

todos_los_locales = pd.DataFrame()

for cp in codigos_postales:
    try:
        lat, lng = get_coordinates(cp)
        lugares = get_places(lat, lng, business_type=tipo_negocio)
        df = clean_data(lugares, cp)
        todos_los_locales = pd.concat([todos_los_locales, df], ignore_index=True)
        print(f"✅ {len(df)} locales encontrados en {cp}")
        time.sleep(1)
    except ValueError as e:
        print(f"⚠️ Error de validación con código postal {cp}: {e}")
    except Exception as e:
        print(f"⚠️ Error inesperado con código postal {cp}: {e}")

# Guardar el CSV generado directamente en la carpeta 'data'
todos_los_locales.to_csv("../data/locales.csv", index=False)

print("Archivo guardado como 'locales.csv' en la carpeta 'data'")

✅ 60 locales encontrados en 28001
✅ 60 locales encontrados en 28002
✅ 60 locales encontrados en 28003
✅ 60 locales encontrados en 28004
✅ 60 locales encontrados en 28005
✅ 60 locales encontrados en 28006
✅ 60 locales encontrados en 28007
✅ 60 locales encontrados en 28008
✅ 60 locales encontrados en 28009
✅ 60 locales encontrados en 28010
✅ 60 locales encontrados en 28011
✅ 60 locales encontrados en 28012
✅ 60 locales encontrados en 28013
✅ 60 locales encontrados en 28014
✅ 60 locales encontrados en 28015
✅ 60 locales encontrados en 28016
✅ 60 locales encontrados en 28017
✅ 60 locales encontrados en 28018
✅ 60 locales encontrados en 28019
✅ 60 locales encontrados en 28020
✅ 60 locales encontrados en 28021
✅ 60 locales encontrados en 28022
✅ 60 locales encontrados en 28023
✅ 60 locales encontrados en 28024
✅ 60 locales encontrados en 28025
✅ 60 locales encontrados en 28026
✅ 60 locales encontrados en 28027
✅ 60 locales encontrados en 28028
✅ 60 locales encontrados en 28029
✅ 60 locales e